# Chapter 4 Sentence Sliding Window Retrieval

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


Reading the database

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["data/人工智能.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

7 

<class 'llama_index.schema.Document'>
Doc ID: b03a0e50-2e8a-49bf-82f0-4a3909364809
Text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 2/13“⼈⼯智能”的各地常⽤名称 中国⼤陆⼈⼯智能 台湾⼈⼯智慧
港澳⼈⼯智能 新⻢⼈⼯智能、⼈⼯智慧 ⽇韩⼈⼯知能 越南智慧⼈造 [展开] [展开] [展开] [展开] [展开] [展开]⼈⼯智能系列内容
主要⽬标 实现⽅式 ⼈⼯智能哲学 历史 技术 术语⼈⼯智能（英语：artiﬁcial intelligence ，缩写为
AI）亦称机器智能，指由⼈制造出来的机器所表现出来的智能。通常⼈⼯
智能是指⽤普通计算机程序来呈现⼈类智能的技术。该词也指出研究这样的智能系统是否能够实现，以及如何实现。同 时，通过 医学 、神经科学
、机器⼈学 及...


In [5]:
from llama_index import SimpleDirectoryReader

documents_en = SimpleDirectoryReader(
    input_files=["data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
print(type(documents_en), "\n")
print(len(documents_en), "\n")
print(type(documents_en[0]))
print(documents_en[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 5ab262c9-a207-4f2d-9513-fc4d5c350cf5
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


Here, the text of each document in documents is concatenated into a string, and then a Document instance is created, which represents the entire document collection.

In [7]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))
document_en = Document(text="\n\n".join([doc.text for doc in documents_en]))

In [8]:
# Replace Chinese punctuation marks with English punctuation marks for easy subsequent processing
# If it is an English document, you can skip this step
# If not processed, it will lead to the inability to correctly segment Chinese sentences, which will affect the size of the subsequent sentence_window and cause the input length to exceed the maximum limit of gpt-3.5-turbo
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')

## 1. Sentence sliding window search settings

A parser object named node_parser is created, the window size is specified as 3, and the original text metadata key is set to ``original_text``. The parser created in this way can be used to extract nodes from text

In [9]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

Define a Chinese text string
Use the get_nodes_from_documents method of node_parser to extract nodes from the provided text.

In [10]:
text = "你好. 你怎么样? 我很好!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [11]:
text_en = "hello. how are you? I am fine!  "

nodes_en = node_parser.get_nodes_from_documents([Document(text=text_en)])

Each individual word

In [12]:
print([x.text for x in nodes])
print([x.text for x in nodes_en])

['你好. ', '你怎么样? ', '我很好!  ']
['hello. ', 'how are you? ', 'I am fine!  ']


Original sentence

In [13]:
print(nodes[1].metadata["window"])
print(nodes_en[1].metadata["window"])

你好.  你怎么样?  我很好!  
hello.  how are you?  I am fine!  


In [14]:
text = "你好. 吧台. 猫狗. 老鼠"
text_en2 = 'hello. bar. cat. dog. mouse.'

nodes = node_parser.get_nodes_from_documents([Document(text=text)])
nodes_en2 = node_parser.get_nodes_from_documents([Document(text=text_en2)])

In [15]:
print([x.text for x in nodes])
print([x.text for x in nodes_en])

['你好. ', '吧台. ', '猫狗. ', '老鼠']
['hello. ', 'how are you? ', 'I am fine!  ']


In [16]:
print(nodes[0].metadata["window"])
print(nodes_en2[0].metadata["window"])

你好.  吧台.  猫狗. 
hello.  bar.  cat. 


### 2.1 Creating an Index

An instance of a language model is created using the `GPT-3.5-turbo` model from `OpenAI`, with the temperature parameter set to 0.1.

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

A `ServiceContext` object is created using the `ServiceContext.from_defaults` method, which contains service-related context information for index construction, including language model, embedding model, and node parser.

In [18]:
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    node_parser=node_parser,
)

sentence_context_en = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

A `VectorStoreIndex` object is created using the `VectorStoreIndex.from_documents` method, which is used to store and retrieve vector information related to documents.

In [19]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

from llama_index import VectorStoreIndex

sentence_index_en = VectorStoreIndex.from_documents(
    [document_en], service_context=sentence_context_en
)

Persist the created index to the specified directory ("./sentence_index") . This allows the index to be reloaded in subsequent runs without having to rebuild it.

In [20]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")
sentence_index_en.storage_context.persist(persist_dir="./sentence_index_en")

Checks if the index file exists and rebuilds it if it does not. If it does exist, it will use the `load_index_from_storage` method to load the index from the existing index file instead of rebuilding it.

In [21]:
# This block of code is optional to check
# if an index file exists, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

if not os.path.exists("./sentence_index_en"):
    sentence_index_en = VectorStoreIndex.from_documents(
        [document_en], service_context=sentence_context_en
    )

    sentence_index_en.storage_context.persist(persist_dir="./sentence_index_en")
else:
    sentence_index_en = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index_en"),
        service_context=sentence_context_en
    )

### 2.2 Post-creation processing

A post-processor instance is created using the `MetadataReplacementPostProcessor` class, and the target metadata key is set to `window`. The function of this post-processor is to replace the content of the target metadata key.

In [22]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

Use the `NodeWithScore` class to associate a score with each node in the original node list to form a node list with scores. 
Use the `deepcopy` function to create a deep copy of the original node list for subsequent comparison.

In [23]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

scored_nodes_en = [NodeWithScore(node=x, score=1.0) for x in nodes_en2]
nodes_old_en = [deepcopy(n) for n in nodes_en2]

In [24]:
print(nodes_old[1].text)
print(nodes_old_en[1].text)

吧台. 
bar. 


Using the `postprocess_nodes` method of the postprocessor, the contents of the target metadata key in the list of nodes with scores are replaced.

In [25]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)
replaced_nodes_en = postproc.postprocess_nodes(scored_nodes_en)

In [26]:
print(replaced_nodes[1].text)
print(replaced_nodes_en[1].text)

你好.  吧台.  猫狗.  老鼠
hello.  bar.  cat.  dog. 


### 2.3 Adding a reordering block

A post-processor instance is created using the `SentenceTransformerRerank` class, the parameter `top_n` is set to 2, and the model used is "BAAI/bge-reranker-base".

In [27]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

A `QueryBundle` object is created containing the query text "I want a dog.". 
A list of two nodes with scores is created, representing text nodes containing the text "This is a cat" and "This is a dog", with scores of 0.6 and 0.4 respectively.

In [28]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("我想要只狗.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="这是只猫"), score=0.6),
    NodeWithScore(node=TextNode(text="这是只狗"), score=0.4),
]

In [29]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query_en = QueryBundle("I want a dog.")

scored_nodes_en = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

Using the `postprocess_nodes` method of the `SentenceTransformerRerank` class, rerank the list of nodes with scores, taking into account the query text. The reranked nodes will be based on the pre-trained sentence transformer model.

In [30]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

reranked_nodes_en = rerank.postprocess_nodes(
    scored_nodes_en, query_bundle=query_en
)

The text and scores in the re-ranked node list are output. This shows the effect of the sentence transformation model on node re-ranking.

In [31]:
print([(x.text, x.score) for x in reranked_nodes])
print([(x.text, x.score) for x in reranked_nodes_en])


[('这是只狗', 0.9660425), ('这是只猫', 0.06396222)]
[('This is a dog', 0.9182736), ('This is a cat', 0.0014040753)]


### 2.4 Running the Index Engine

Use the `as_query_engine` method to convert `sentence_index` to a query engine object `sentence_window_engine`. 
Here, the `top k` of `similarity` is set to 6, and the `node_postprocessors` parameter is passed in, which contains the `postproc` and `rerank` postprocessors created previously.

In [32]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

sentence_window_engine_en = sentence_index_en.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

The query engine's `query` method executes a query, "What is the key to success in the field of artificial intelligence?" The query engine will use the previously set post-processor to post-process the node.

In [33]:
window_response = sentence_window_engine.query(
    "在人工智能领域建功立业的关键是什么?"
)

In [34]:
window_response_en = sentence_window_engine_en.query(
    "What are the keys to building a career in AI?"
)

The `display_response` function provided by the `LLAMA` framework displays the response results of the query. This usually includes a set of nodes that match the query, as well as their text, scores, and other information. 
This way, the results of the query can be better visualized and understood in the `Notebook` environment.

In [35]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** 在人工智能领域建功立业的关键是系统能够正确解释外部数据，从中学习，并利用这些知识通过灵活适应实现特定目标和任务的能力。

In [36]:
from llama_index.response.notebook_utils import display_response

display_response(window_response_en)

**`Final Response:`** Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.

## 2. Merge the above operations

`documents`: list of documents to build index for. 
`llm`: OpenAI language model instance. 
`embed_model`: name or path to embedding model. 
`sentence_window_size`: size of sentence window. 
`save_dir`: directory to persist index. 

Create a node_parser for sentence window. 
Create a ServiceContext with context information such as language model and node_parser. 
If index does not exist in the specified directory, create a VectorStoreIndex based on the provided documents and persist it to the specified directory. 
If index file already exists in the directory, load index from file. 
Return the constructed sentence window index.

In [37]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
# create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

def build_sentence_window_index_en(
    documents_en,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_en",
):
# create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index_en = VectorStoreIndex.from_documents(
            documents_en, service_context=sentence_context
        )
        sentence_index_en.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index_en = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index_en


`sentence_index`: constructed sentence window index. 
`similarity_top_k`: top k for similarity query. 
`rerank_top_n`: reranked top n. 

Defines two postprocessors: `postproc` for replacing metadata keys, `rerank` for reranking nodes using sentence transformation model. 
Creates a query engine `sentence_window_engine`, converts sentence window index to query engine, and uses defined postprocessors. 
Returns the constructed query engine.

In [38]:
def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
# define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

def get_sentence_window_query_engine_en(
    sentence_index_en, similarity_top_k=6, rerank_top_n=2
):
# define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine_en = sentence_index_en.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine_en

Call the previously defined `build_sentence_window_index` function, passing in the document list, language model instance, and save directory to build the sentence window index.

In [39]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)

index_en = build_sentence_window_index_en(
    [document_en],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index_en",
)



Call the previously defined `get_sentence_window_query_engine` function, passing in the constructed sentence window index and similarity `top k` to get the query engine for the sentence window. 
Here, `similarity_top_k` is set to 6.

In [40]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)
query_engine_en = get_sentence_window_query_engine(index_en, similarity_top_k=6)

## 3. TruLens Evaluation

Read the generated questions from the file named 'generated_questions.text' and store them in the `eval_questions` list.

In [41]:
eval_questions = []
with open('data/generated_questions.txt', 'r') as file:
    for line in file:
# Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

eval_questions_en = []
with open('data/generated_questions_en.txt', 'r') as file:
    for line in file:
# Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

A function `run_evals` is defined that accepts the generated list of questions, the `TruLens` recorder, and the query engine as parameters. For each question, a recording is started using the `TruLens` recorder, and then the query is executed using the query engine.

In [42]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)


def run_evals_en(eval_questions_en, tru_recorder, query_engine):
    for question in eval_questions_en:
        with tru_recorder as recording:
            response = query_engine.query(question)

Reset the `TruLens` database using the `reset_database` method of the Tru class.

In [43]:
from utils import get_prebuilt_trulens_recorder

from trulens_eval import Tru

tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### 3.1 Set the sliding window size to 1

Call the previously defined functions `build_sentence_window_index` and `get_sentence_window_query_engine` to build the sentence window index and query engine respectively. Here, the window size is set to 1, and the save directory is specified as "sentence_index_1".

In [44]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",  # "local:BAAI/bge-small-en-v1.5" for english
    sentence_window_size=1,
    save_dir="sentence_index_1",
)
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)


In [45]:
sentence_index_1_en = build_sentence_window_index_en(
    documents_en,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",  # "local:BAAI/bge-small-en-v1.5" for english
    sentence_window_size=1,
    save_dir="sentence_index_1_en",
)
sentence_window_engine_1_en = get_sentence_window_query_engine(
    sentence_index_1_en
)
tru_recorder_1_en = get_prebuilt_trulens_recorder(
    sentence_window_engine_1_en,
    app_id='sentence window engine 1_en'
)

Call the previously defined evaluation function `run_evals`, pass in the generated question list, `TruLens` recorder `tru_recorder_1` and the constructed query engine `sentence_window_engine_1`, and run the evaluation task.

In [46]:
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)
run_evals(eval_questions_en, tru_recorder_1, sentence_window_engine_1_en)

openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-me3Y2JVoMQFvYW4UUurcFXXM on tokens per min (TPM): Limit 60000, Used 58250, Requested 1999. Please try again in 249ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-me3Y2JVoMQFvYW4UUurcFXXM on tokens per min (TPM): Limit 60000, Used 59832, Requested 502. Please try again in 334ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organizati

In [47]:
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1'
)

tru_recorder_1_en = get_prebuilt_trulens_recorder(
    sentence_window_engine_1_en,
    app_id='sentence window engine 1_en'
)

View Results

In [48]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [49]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_87b8d0d554e7d74fa19c16f4692d69cf,"""\u4eba\u5de5\u667a\u80fd\u4e2d\u7684\u5148\u9...","""\u5148\u9a8c\u77e5\u8bc6\u5728\u4eba\u5de5\u6...",-,"{""record_id"": ""record_hash_87b8d0d554e7d74fa19...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-12T10:30:49.113462"", ""...",2024-03-12T10:31:02.269094,0.9,0.40,1.000000,"[{'args': {'prompt': '人工智能中的先验知识是如何被存储的？', 're...","[{'args': {'prompt': '人工智能中的先验知识是如何被存储的？', 're...","[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",13,0,0.0
1,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b9425d9aa02130eec6c73f7cc6f700f8,"""\u4eba\u5de5\u667a\u80fd\u7684\u81ea\u6211\u6...","""The self-updating and self-improving capabili...",-,"{""record_id"": ""record_hash_b9425d9aa02130eec6c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-12T10:31:02.914647"", ""...",2024-03-12T10:31:09.293573,1.0,0.60,0.800000,[{'args': {'prompt': '人工智能的自我更新和自我提升是否可能导致其脱离人...,[{'args': {'prompt': '人工智能的自我更新和自我提升是否可能导致其脱离人...,"[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",6,0,0.0
2,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8c266922b3864f14c5aa3fd4fc98923c,"""\u7ba1\u7406\u8005\u5982\u4f55\u7ba1\u7406AI\...","""Management should consider adjusting their wo...",-,"{""record_id"": ""record_hash_8c266922b3864f14c5a...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-12T10:31:09.505494"", ""...",2024-03-12T10:31:12.333074,0.8,0.25,1.000000,"[{'args': {'prompt': '管理者如何管理AI？', 'response':...","[{'args': {'prompt': '管理者如何管理AI？', 'response':...",[{'args': {'source': '任何的科技都会有瓶颈， 摩尔定律 到⽬前也遇到相...,2,0,0.0
3,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_48957156666710cd55d5059c9ed69b56,"""\u5f3a\u4eba\u5de5\u667a\u80fd\u662f\u4ec0\u4...","""\u5f3a\u4eba\u5de5\u667a\u80fd\u662f\u4e00\u7...",-,"{""record_id"": ""record_hash_48957156666710cd55d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-12T10:31:12.872050"", ""...",2024-03-12T10:31:21.471832,0.8,0.80,0.333333,"[{'args': {'prompt': '强人工智能是什么？', 'response': ...","[{'args': {'prompt': '强人工智能是什么？', 'response': ...",[{'args': {'source': 'The Behavioral and Brain...,8,0,0.0
4,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_c3d563072ae3ef443e6e102c53e1677e,"""\u4eba\u5de5\u667a\u80fd\u88ab\u6ee5\u7528\u5...","""The misuse of artificial intelligence can lea...",-,"{""record_id"": ""record_hash_c3d563072ae3ef443e6...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-12T10:31:21.670386"", ""...",2024-03-12T10:31:28.646002,0.9,0.50,1.000000,"[{'args': {'prompt': '人工智能被滥用带来的危害？', 'respons...","[{'args': {'prompt': '人工智能被滥用带来的危害？', 'respons...","[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",6,0,0.0


### 3.2 Set the sliding window size to 3

The previously defined function is called to build the sentence window index, query engine, and `TruLens` recorder. The window size is set to 3, and the save directory is specified as "sentence_index_3".

In [ ]:
sentence_index_3 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",  # "local:BAAI/bge-small-en-v1.5" for english
    sentence_window_size=3,
    save_dir="sentence_index_3",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)

In [ ]:
sentence_index_3_en = build_sentence_window_index_en(
    documents_en,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",  # "local:BAAI/bge-small-en-v1.5" for english
    sentence_window_size=3,
    save_dir="sentence_index_3_en",
)
sentence_window_engine_3_en = get_sentence_window_query_engine(
    sentence_index_3_en
)

tru_recorder_3_en = get_prebuilt_trulens_recorder(
    sentence_window_engine_3_en,
    app_id='sentence window engine 3_en'
)

Call the `run_evals` function, passing in the generated question list `eval_questions`, the `TruLens` recorder `tru_recorder_3`, and the constructed query engine `sentence_window_engine_3` to run the evaluation task.

In [ ]:
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)
run_evals(eval_questions_en, tru_recorder_3_en, sentence_window_engine_3_en)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [ ]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e9815da1c66c0943f4d155a06f94e9c4,"""\u4eba\u5de5\u667a\u80fd\u4e2d\u7684\u5148\u9...","""\u5148\u9a8c\u77e5\u8bc6\u5728\u4eba\u5de5\u6...",-,"{""record_id"": ""record_hash_e9815da1c66c0943f4d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-10T22:55:51.162029"", ""...",2024-03-10T22:56:03.820730,0.9,0.4,1.0,"[{'args': {'prompt': '人工智能中的先验知识是如何被存储的？', 're...","[{'args': {'prompt': '人工智能中的先验知识是如何被存储的？', 're...","[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",12,0,0.0
1,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_140932cb020d95e0554ecf0489eb42f2,"""\u4eba\u5de5\u667a\u80fd\u7684\u81ea\u6211\u6...","""The self-updating and self-improving capabili...",-,"{""record_id"": ""record_hash_140932cb020d95e0554...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-10T22:56:04.085254"", ""...",2024-03-10T22:56:10.348414,1.0,0.5,0.8,[{'args': {'prompt': '人工智能的自我更新和自我提升是否可能导致其脱离人...,[{'args': {'prompt': '人工智能的自我更新和自我提升是否可能导致其脱离人...,"[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",6,0,0.0
2,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5bd9fa7d1b1997c136b9d1d4ce3d2684,"""\u7ba1\u7406\u8005\u5982\u4f55\u7ba1\u7406AI\...","""Management should consider adjusting their wo...",-,"{""record_id"": ""record_hash_5bd9fa7d1b1997c136b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-10T22:56:10.552787"", ""...",2024-03-10T22:56:13.694339,0.8,0.3,0.9,"[{'args': {'prompt': '管理者如何管理AI？', 'response':...","[{'args': {'prompt': '管理者如何管理AI？', 'response':...",[{'args': {'source': '任何的科技都会有瓶颈， 摩尔定律 到⽬前也遇到相...,3,0,0.0
3,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_cd9a2aa2bdf60288d1b04cdbeac630f3,"""\u5f3a\u4eba\u5de5\u667a\u80fd\u662f\u4ec0\u4...","""\u5f3a\u4eba\u5de5\u667a\u80fd\u662f\u4e00\u7...",-,"{""record_id"": ""record_hash_cd9a2aa2bdf60288d1b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-10T22:56:13.882046"", ""...",2024-03-10T22:56:21.948222,0.9,0.7,1.0,"[{'args': {'prompt': '强人工智能是什么？', 'response': ...","[{'args': {'prompt': '强人工智能是什么？', 'response': ...",[{'args': {'source': 'The Behavioral and Brain...,8,0,0.0
4,sentence window engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_736205619e133e5333d36a19a29293fe,"""\u4eba\u5de5\u667a\u80fd\u88ab\u6ee5\u7528\u5...","""The misuse of artificial intelligence can lea...",-,"{""record_id"": ""record_hash_736205619e133e5333d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-03-10T22:56:22.140156"", ""...",2024-03-10T22:56:28.835570,0.9,0.3,0.0,"[{'args': {'prompt': '人工智能被滥用带来的危害？', 'respons...","[{'args': {'prompt': '人工智能被滥用带来的危害？', 'respons...","[{'args': {'source': '2/2/24, 2:43 PM ⼈⼯智能 - ...",6,0,0.0
